In [80]:
import numpy as np
import pandas as pd
import requests
from lxml import html
import time
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from collections import defaultdict
import sys
import zlib

In [4]:
song_meta = pd.read_json('song_meta.json')

In [5]:
song_meta.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [8]:
song_meta['artist_name_basket'] = song_meta['artist_name_basket'].apply(lambda x : " ".join(x))

In [13]:
album_info = song_meta[['album_id', 'album_name','artist_name_basket']]

In [14]:
album_info = album_info.drop_duplicates().reset_index(drop= True)

In [16]:
album_info['keyword'] = album_info['album_name'] + " " + album_info['artist_name_basket'] 

In [155]:
# keyword로 검색 진행
album_info[['keyword']]

,keyword
0,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트 Various Artists
1,"Bach : Partitas Nos. 2, 3 & 4 Murray Perahia"
2,Hit Peter Gabriel
3,Feeling Right (Everything Is Nice) (Feat. Popc...
4,그남자 그여자 Jude Law
...,...
351383,Peppermint Tiffany Young
351384,My Blood Westlife
351385,생일왕국의 프린세스 프링 OST1 : 프린세스 프링의 초대 ButterFly
351386,The Best Best Of The Black President Fela Kuti


# 크롤링

In [163]:
def search_keyword(keyword):
    url = 'https://www.google.co.kr/search?q='+ keyword +'&newwindow=1&hl=ko&tbm=isch&source=hp&biw=1920&bih=969'
    driver.switch_to.window(driver.window_handles[0])
    driver.get(url)
    
    return driver

In [167]:
start_idx = 0
url_dict = defaultdict(str)
save_point = 10000
all_album_cnt = album_info.shape[0]
with open('end_here.txt', 'w', encoding = 'utf-8') as f:
    f.write('')
    
driver = webdriver.Chrome("C://chromedriver/chromedriver.exe")
keywords = album_info.iloc[start_idx:]['keyword'].tolist()

for cnt ,(i ,keyword) in enumerate(zip(range(start_idx, all_album_cnt), keywords)):
    driver = search_keyword(keyword)
    driver.implicitly_wait(5)
    
    #img 주소를 가져온다.
    try:
        img = driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]/img')
        link = img.get_attribute('src')
        url_dict[i] = link
    except Exception as e:
        f = open('end_here.txt', 'a', encoding='utf-8')
        f.write('{}\n'.format(i))
#         df = pd.DataFrame(url_dict.values(),index = url_dict.keys(), columns = ['url'])
#         df.to_csv(f'url_set_errorend_{i}.csv')
        continue
    print(f'{i} done')
        
    #save_point 마다 저장한다.
    if cnt != 0 and cnt % save_point == 0:
        df = pd.DataFrame(url_dict.values(),index = url_dict.keys(), columns = ['url'])
        df.to_csv(f'url_set_{int(cnt / save_point)}.csv')
        url_dict = defaultdict(str)
    # 나머지 저장
    elif cnt == (all_album_cnt -start_idx - 1):
        df = pd.DataFrame(url_dict.values(),index = url_dict.keys(), columns = ['url'])
        df.to_csv(f'url_set_{ cnt // save_point + 1}.csv')
        del url_dict
        f.close()
    time.sleep(1)
f.close()

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done


WebDriverException: Message: chrome not reachable
  (Session info: chrome=93.0.4577.82)
